In [1]:
import pennylane as qml
from pennylane import numpy as np

## V.3.1a

In [ ]:
n_wires = 1
dev = qml.device("default.qubit", wires=n_wires)
p=3
cost_h=qml.Hamiltonian([1], [qml.PauliZ(0)])
mixer_h=qml.Hamiltonian([1], [qml.PauliX(0)])
params=np.ones((p, 2),requires_grad=True) * 0.5

def qaoa_layer(params,cost_h):
    """Implement one QAOA layer alternating H_C and H_M.

    Args:
        params (np.array): An array with the trainable parameters of the QAOA ansatz.
        cost_h (qml.Hamiltonian): The cost Hamiltonian
    """  
    ##################
    # YOUR CODE HERE #
    ##################
    qml.qaoa.cost_layer(params[0], cost_h)
    qml.qaoa.mixer_layer(params[1], mixer_h)
    
def qaoa_circuit(params,p,cost_h):
    """Implement the initial state and p layers of the QAOA ansatz.

    Args:
        params (np.array): An array with the trainable parameters of the QAOA ansatz.
        p (int): Number of layers of the QAOA ansatz.
        cost_h (qml.Hamiltonian): The cost Hamiltonian
    """  
    ##################
    # YOUR CODE HERE #
    ##################
    for w in range(n_wires):
        qml.Hadamard(wires=w)
    qml.layer(qaoa_layer, p, params, cost_h)

@qml.qnode(dev)
def probability_circuit(params,p,cost_h):
    """QAOA circuit which returns the probabilities.
    Args:
        params (np.array): An array with the trainable parameters of the QAOA ansatz.
        p (int): Number of layers of the QAOA ansatz.
        cost_h (qml.Hamiltonian): The cost Hamiltonian
    Returns:
        (np.tensor): A tensor with the probabilities of measuring the quantum states.
    """  
    ##################
    # YOUR CODE HERE #
    ##################
    qaoa_circuit(params, p, cost_h)
    return qml.probs(wires=range(n_wires))

## V.3.1b

In [ ]:
n_wires = 1
dev = qml.device("default.qubit", wires=n_wires)
p=3
cost_h=qml.Hamiltonian([1], [qml.PauliZ(0)])
mixer_h=qml.Hamiltonian([1], [qml.PauliX(0)])
params=np.ones((p, 2),requires_grad=True) * 0.5

def optimizer(params,p,cost_h):
    """Optimizer to adjust the parameters.

    Args:
        params (np.array): An array with the trainable parameters of the QAOA ansatz.
        p (int): Number of layers of the QAOA ansatz.
        cost_h (qml.Hamiltonian): The cost Hamiltonian
        
    Returns:
        (np.array): An array with the optimized parameters of the QAOA ansatz.
    """      
    optimizer = qml.AdamOptimizer(0.1)
    steps = 100
    for _ in range(steps):
        params, _ , _ = optimizer.step(cost_function, params,p,cost_h)
    return params

@qml.qnode(dev)
def cost_function(params,p,cost_h):
    """Cost function of the QAOA circuit.
    Args:
        params (np.array): An array with the trainable parameters of the QAOA ansatz.
        p (int): Number of layers of the QAOA ansatz.

    Returns: 
        (np.tensor): A 1 dimensional tensor with the expected value of the cost Hamiltonian after applying the QAOA circuit.
    """  
    ##################
    # YOUR CODE HERE #
    ##################
    qaoa_circuit(params, p, cost_h)
    return qml.expval(cost_h)

def execute_QAOA(params,p,cost_h):
    """Execute QAOA Algorithm.
    Args:
        params (np.array): An array with the trainable parameters of the QAOA ansatz.
        p (int): Number of layers of the QAOA ansatz.
        cost_h (qml.Hamiltonian): The cost Hamiltonian

    Returns:
        (np.tensor): A tensor with the final probabilities of measuring the quantum states.
        (np.array): The optimized parameters of the QAOA.
    """  
    ##################
    # YOUR CODE HERE #
    ##################
    params = optimizer(params, p, cost_h)
    probs = probability_circuit(params, p, cost_h)
    return probs, params



## V.3.2

In [ ]:
n_particles = 3
n_wires = n_particles
h_list=[1,2,-3]
# Define the mixer Hamiltonian
mixer_h = qml.Hamiltonian([1.0] * n_wires, [qml.PauliX(i) for i in range(n_wires)])
# Define number of layers and initial parameters
p=5
params=np.ones((p, 2),requires_grad=True) * 0.5

def build_cost_ising(n_particles, h_list):
    """Function to build the cost Hamiltonian of the Longitudinal Ising problem.
    
    Args:
        n_particles (int): Number of particles to be considered in the Hamiltonian.
        h_list (list[float]): List of magnetic field values for each particle.
        
    Returns:
        (qml.Hamiltonian): The cost Hamiltonian of the Longitudinal Ising model.
    """
    ##################
    # YOUR CODE HERE #
    ##################
    interactions_coeffs = [-1] * (n_particles - 1)
    background_coeffs = [-hi for hi in h_list]
    coeffs = interactions_coeffs + background_coeffs
    interactions = [qml.Z(i) @ qml.Z(i + 1) for i in range(n_particles - 1)]
    background = [qml.Z(i) for i, _ in enumerate(h_list)]
    component_Hs = interactions + background
    return qml.Hamiltonian(coeffs, component_Hs)

def QAOA_ising(params,p,cost_h):
    """QAOA Algorithm to solve the Longitudinal Ising problem.
    Args:
        params (np.array): an array with the trainable parameters of the QAOA ansatz.
        p (int): number of layers of the QAOA ansatz.
        cost_h (qml.Hamiltonian): the cost Hamiltonian.
        
    Returns:
        (np.tensor): a tensor with the final probabilities of measuring the quantum states.
        (np.array): the optimized parameters of the QAOA.
    """  
    ##################
    # YOUR CODE HERE #
    ##################
    params = optimizer(params, p, cost_h)
    probs = probability_circuit(params, p, cost_h)
    return probs, params
    


## V.3.3

In [ ]:
numbers = [1,3,4,7,13]
total_sum = sum(numbers)
# Number of qubits needed (each number is represented by a qubit)
n_wires = len(numbers)
# Define the mixer Hamiltonian
mixer_h = qml.Hamiltonian([1.0] * n_wires, [qml.PauliX(i) for i in range(n_wires)])
dev = qml.device('default.qubit', wires=n_wires)
# Define the hyperparameters for the optimizer:
p=20
params=np.ones((p, 2),requires_grad=True) * 0.5
step_size=0.001
max_steps=150

def build_cost_number_partition(numbers):
    """Function to build the cost Hamiltonian of a number partition problem.

    Args:
        numbers (list): A list with the numbers we want to divide in two groups with equal sums.
        
    Returns:
        (qml.Hamiltonian): The cost Hamiltonian of the number partition problem
    """      
    ##################
    # YOUR CODE HERE #
    ##################
    coeffs = [ni * nj for ni in numbers for nj in numbers]
    components = [qml.Z(i) @ qml.Z(j) for i in range(len(numbers)) for j in range(len(numbers))]
    return qml.Hamiltonian(coeffs, components)
    
def optimizer_number_partition(params,p,step_size,max_steps,cost_h):
    """Optimizer to adjust the parameters.

    Args:
        params (np.array): An array with the trainable parameters of the QAOA ansatz.
        p (int): Number of layers of the QAOA ansatz.
        step_size (float): Learning rate of the gradient descent optimizer
        max_steps (int): Number of iterations of the optimizer.
        cost_h (qml.Hamiltonian): The cost Hamiltonian.

    Returns:
        (np.array): An array with the optimized parameters of the QAOA ansatz.
    """      
    optimizer = qml.AdamOptimizer(step_size) 
    for _ in range(max_steps):
        params, _,_ = optimizer.step(cost_function, params,p,cost_h)
    return params

def QAOA_number_partition(params,p,step_size,max_steps,cost_h):
    """QAOA Algorithm to solve the number partition problem.

    Args:
        params (np.array): An array with the trainable parameters of the QAOA ansatz.
        p (int): Number of layers of the QAOA ansatz.
        step_size (float): Learning rate of the gradient descent optimizer
        max_steps (int): Number of iterations of the optimizer.
        cost_h (qml.Hamiltonian): The cost Hamiltonian.

    Returns:
        (np.tensor): A tensor with the final probabilities of measuring the quantum states.
        (np.array): The optimized parameters of the QAOA.
    """  
    ##################
    # YOUR CODE HERE #
    ##################
    params = optimizer_number_partition(params, p, step_size, max_steps, cost_h)
    probs = probability_circuit(params, p, cost_h)
    return probs, params